In [7]:
%load_ext autoreload
%autoreload 2
import manenv
from manenv.utils.product_utils import *
from manenv.core.world import World
from manenv.core.component import *
from manenv.render_wrapper import RenderWrapper
from manenv.core.product import *
from manenv.core.effector import *

import manenv.effectors as eff
import manenv.components as cmp


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
world = World((3, 3))
s = np.zeros((5, 5), dtype=np.int8)
s[1][2] = 2
s[1][1] = 2
s[2][1] = 1
s[3][1] = 1
product = Product(s)
world.place_component(make_vector(1, 1), cmp.Conveyor(
    np.array([
    [0, 0, 0],
    [1, 0, -1],
    [0, 0, 0]
])))
world.place_component(make_vector(2, 0), cmp.Conveyor(
    np.array([
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, 0] 
])))
world.place_component(make_vector(2, 1), cmp.Outport())

grabber = eff.Grabber(make_vector(0, 2))
buddy = eff.Grabber(make_vector(1, 3))
welder = eff.Welder(make_vector(1, 2))
transformer = eff.Transformer(1, 2, make_vector(2,  4))
discard = eff.Acceptor(make_vector(1, 7))

world.place_component(make_vector(1, 2), cmp.Spawner(product))
assembler : cmp.Assembler = cmp.Assembler(make_vector(10, 10), [grabber, buddy, welder, discard, transformer])
world.place_component(make_vector(1, 0), assembler)
wrapper = RenderWrapper(world)

In [9]:
wrapper.reset()
assembler.place_in_inventory(product.copy())
assembler.place_in_inventory(product.copy())
world.update()

grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
buddy.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
world.update()

grabber.set_action(eff.GrabberActions.RELEASE.value)
buddy.set_action(eff.GrabberActions.RELEASE.value)
world.update()

welder.set_action(eff.WelderActions.WELD_EAST.value)
world.update()
world.update()

grabber.set_action(eff.GrabberActions.GRAB.value)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
world.update()

grabber.set_action(eff.GrabberActions.RELEASE.value)
world.update()

transformer.set_action(eff.TransformerActions.TRANSFORM.value)
world.update()
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
world.update()
world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
# world.update()


In [10]:
wrapper.render()

# Gym Environment Test

In [11]:
from manenv.gym_wrapper import MARLFactoryEnvironment

In [12]:
world.reset()
environment = MARLFactoryEnvironment(world)
renderer = RenderWrapper(environment._world)

In [13]:
for _ in range(100):
    action = environment.action_space.sample()
    environment.step(action)
renderer.render()

Exception: Not in bounds